In [1]:
import time
from datetime import datetime

import pandas as pd
from database import Database
from sklearn.pipeline import Pipeline
from sqlalchemy.dialects.postgresql import insert
from ssrs import BookingData
from transformers import BookingDataEncoder, BookingDataReadCsv

In [3]:
db = Database()

reservation = db.gwg_reservation

/home/jovyan/work/database.py:34: SAWarning: Did not recognize type 'geometry' of column 'geom'
  Table(


In [ ]:
date_fr = "11/01/2021"
date_to = "12/31/2026"